<a href="https://colab.research.google.com/github/dobriyaladitya/blood_noBlood/blob/master/adi_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facial blood detection using CNN

Note: Make sure you have python installed in order to work the pip command. The method is implemented using keras, that needs to be pre installed as well. 

Implementation is done in 4 stages:


*   Pre-processing images
*   Defining model architecture
*   Training the model
*   Measuring the performance of the model created

To do so we first import all necesarry libraries and packages from keras,


In [0]:
"""
Implementing convolutional neural networks

"""
!pip install tensorflow
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

INPUT_DIR_TRAIN = "/data/training_set" #change to location of the folder on drive
INPUT_DIR_TEST = "/data/test_set" #change to location of the folder on drive
NUM_OF_EPOCH = 10


The next part would be defining a function for preparing the training data, to further enhance accuracy and reduce chances of overfitting we use data augmentation

In [0]:
def prepare_training_data():

    # shear_range is for randomly applying shearing transformations
    # zoom_range is for randomly zooming inside pictures
    # horizontal_flip is for randomly flipping half the images horizontally—relevant when there are no assumptions of horizontal asymmetry
    # fill_mode is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift
    train_datagen_augmented = ImageDataGenerator(rescale = 1./255, shear_range = 0.1, zoom_range = 0.2, horizontal_flip = True, fill_mode= 'nearest')

    train_img_data_gen = ImageDataGenerator(rescale = 1./255)
    test_img_data_gen = ImageDataGenerator(rescale = 1./255)

    training_set = train_datagen_augmented.flow_from_directory(INPUT_DIR_TRAIN , target_size = (150, 150), batch_size = 32, class_mode = 'binary')
    testing_set = test_img_data_gen.flow_from_directory(INPUT_DIR_TRAIN , target_size = (150, 150), batch_size = 32, class_mode = 'binary')
    return training_set, testing_set


In [0]:
# Displayes the image along with the label. Can be used to verify the input data
def show_sample_data_with_labels(training_set, range_vl):
    import matplotlib.pyplot as plt
    x,y = training_set.next()
    for i in range(0,range_vl):
        image = x[i]
        label = y[i]
        print (label)
        plt.imshow(image)
        plt.show()



Now for the heart of the network, we define the model structure. We use a set of input layers and a hidden and output layer as highlighted in the code below. A brief understanding of the layer types, 



*   Convolutional layers are image feature detectors, they help the machine identify certain features in an image that help it differentiate
*   Pooling layers are used to reduce the size of feature map, making the model more robust without loosing any of the features due to down sampling, we have used max pooling in our implementation
*   The flatter layer transfors our 2D array into a vector (1D tensor)
*   Dropout is as the word reads, used to drop nodes randomly from a few layers of the network, again to account for overfitting and also reducing training time for each epoch

All of the layers have ReLU activation except the output layer which is using sigmoid, this is done to ensure that the final output of the model comes out to a value between 0 and 1, a standard procedure for most classification supervised learning problems





In [0]:
def define_the_model():
    model = Sequential()
    # input_shape (image_height, image_width,image_channels)=(150, 150, 3) where 150*150 is the image size and 3 is the RGB elements
    model.add(Convolution2D(filters = 32, kernel_size = (3,3), input_shape=(150,150,3), activation = 'relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(units = 512, activation = 'relu')) #hidden layer
    model.add(Dense(units = 1, activation = 'sigmoid')) #output layer
    return model


A function to plot the graph and return the accuracy and loss of the training as well as validation or testing set,

In [0]:
def plot_the_graph(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()


Calling all the functions to execute the model and compile a resulting graph that gives us the loss and accuracy visualisation, 

In [0]:
# Prepares the training and test data
# training_set,test_set = prepare_training_data()
training_set, test_set = prepare_training_data()

# Defines the model with layers
model = define_the_model()

# Selects the optimizer, validation function and the metrics
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

print(model.summary())
model.save('blooded_or_not_2')

# Processes in the batches
history = model.fit_generator(training_set, steps_per_epoch=100, nb_epoch = NUM_OF_EPOCH, validation_data = test_set, validation_steps = 50)

In [0]:
x,y = next(test_set)
predictions = model.predict(x)
for i in range (0,len(predictions)):
    print(i,': ',y[i], '    ', predictions[i])

#plot_the_graph(history)

0 :  1.0      [1.]
1 :  0.0      [1.]
2 :  1.0      [1.]
3 :  0.0      [1.]
4 :  0.0      [1.]
5 :  1.0      [1.]
6 :  0.0      [1.]
7 :  1.0      [1.]
8 :  0.0      [1.]
9 :  0.0      [1.]
10 :  0.0      [1.]
11 :  1.0      [1.]
12 :  1.0      [1.]
13 :  0.0      [1.]
14 :  0.0      [1.]
15 :  0.0      [1.]
16 :  0.0      [1.]
17 :  1.0      [1.]
18 :  1.0      [1.]
19 :  1.0      [1.]
20 :  0.0      [1.]
21 :  0.0      [1.]
22 :  1.0      [1.]
23 :  0.0      [1.]


![alt text](https://drive.google.com/uc?id=1XgH5CkmxbVWffDlRzqehCwhXL95HMwip)

![alt text](https://drive.google.com/uc?id=1jkhG_ijZN6mHA1vGlNqRxSErlPnX-Z__)

As you can see the accuracy of the model increases with the epochs and the loss comes down as we move from 2 to 10 epochs.